# LangChain Introduction

LangChain 是圍繞 LLMs 構建的框架。我們可以將其用於聊天機器人、Question-Answering (QA)、摘要等等。

這個函式庫的核心思想是我們可以將不同的元件 “鏈結” 在一起，以創建更多元的 LLMs 應用。 Chain 來自幾個 Module 的多個組件：

1. **Prompt templates**：Prompt templates 是不同類型提示的範本。例如「 chatbot 」樣式模板、ELI5 問答等
2. **LLMs**：像 GPT-3、Mistral、Llama、Breeze、TAIDE 等大型語言模型
3. **Agents**：Agents 使用 LLMs 決定應採取的動作。可以使用網路搜尋(Google Search)或計算器(Python func)之類的工具，並將所有工具包裝成一個邏輯循環的操作。
4. **Memory**：短期記憶、長期記憶。

我們將從 Prompt templates 和 LLMs 的基礎知識開始。以下教學將提供兩個 LLMs 選項，包含 Hugging Face Hub 或 Hugging Face Pipeline 的模型。

- GitHub: https://github.com/langchain-ai/langchain
- Docs: https://python.langchain.com/en/latest/index.html

In [ ]:
!pip install langchain
!pip install langchain_huggingface
!pip install bitsandbytes
!pip install accelerate

In [ ]:
import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.schema import StrOutputParser
from langchain_huggingface import ChatHuggingFace

In [ ]:
from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

### 1. LLMs
透過 Langchain 載入 Huggingface 上的各種大型語言模型，在 Langchain 內模型可以分為

1. LLM 模式：給予文字輸入，然後文字輸出

2. Chat Models 模式：基於LLM模式的更進階的模式，他的輸入和輸出是格式化的chat messages

In [ ]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = """
提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: """
print(llm(prompt))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: 2022 年全球最賣座的電影是《黑亞當：黑寡婦傳奇》(Black Adam: Black Superhero Legend)。


In [ ]:
prompt = """
提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: """
print(llm(prompt))


提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: 目前尚未決定，需待 NBA 2023 賽季進行後結果。


### 2. Prompt
一個好的 Prompt 通常包含以下四個組成部分：

1. **指示**: 告訴模型要做什麼，如何使用提供的信息，如何處理查詢，並建立輸出
2. **範例輸入**: 提供範例輸入，以向模型示範預期的內容
3. **範例輸出**: 提供對應的範例輸出
4. **查詢**: 您希望模型處理的實際輸入

以下介紹幾種在 LangChain 上使用 Pormpt 的方式

### 2.1 PromptTemplate

In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我一個笑話")
message = prompt_template.format()

# 模型生成
print(llm(message))

告訴我一個笑話

1. 在美國，如果你開車的速度超過了時速80公里，警察會追你。但是，如果你的車是紅色的，警察還會用閃光燈警告你減速。


In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我關於一個{content}的{adjective}笑話。")
message = prompt_template.format(adjective="悲傷的", content="數據科學家")

# 模型生成
print(llm(message))

告訴我關於一個數據科學家的悲傷的笑話。

在一個小村莊里，一位年輕的數據科學家被任命為村长。他非常努力地工作，希望能改善村民們的生活。有一天，他收到了一份報告，報告中提到：“如果我們在村子里的每个家庭都建一个水井，那么村民们可以更容易地獲得水源。”

數據科學家很感兴趣，于是决定去考察一下这个想法。他遍访了全村，了解了村民们对这一提案的看法。大部分人表示支持，但也有一些人提出了担忧。其中有两个老人家说道：“我们老一輩的人并没有什么问题，我们可以用现有的水源。你为什么要给新一代人带来麻煩呢？”

數據科学家想了想，觉得他们的观点很有道理。他决定先试着用现有的水源解决村民们的需求。他向政府请示，得到批准后，开始了项目。

数月后，数据科学家回到村庄，发现村民们的生活条件有所改变。水井已经建成，但很多人还是会出现缺水的情况。他问村民们原因，村民们回答说：“由于雨水少，河流干渴，所以水源不足。”

数据科学家又去考察了一下，确认了村民们的描述。他决定再找其他方法解決问题。他再次向政府请示，得到同意后，开始了新的计划。

数年过去了，村民们的生活条件依然没有太大改变。数据科学家感到很沮喪，他决定告诉村民们真相。他跟村民们讲述了他的故事，以及如何应对缺水等问题。

村民们听完之后，有些人表示理解，但有些人仍然不满。其中一名老年人说道：“为什么你一直在寻求外帮助，而不是尝试利用我们自己的資源？”

数据科学家深有感触，他决定重新思考问题。他回到村庄，与村民们一起研究如何利用自然资源，如雨雪、冰層和地下水。在一段时间内，他和村民们共同研制了一套有效的水利管理方案。

最后，村民们的生活条件有了很大改变。水井不再需要，河流也恢复了生命力。村民们感激不已，感謝数据科学家的努力。

从这个故事中，我们可以看到，数据科学家的努力并非白费。他终于找到了适合村庄的解決方案，使村民们的生活条件有所改变。


### 2.2 ChatPromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個擁有強大能力的 AI 機器人。你的名字是{name}。"),
        ("human", "你好，你好嗎？"),
        ("ai", "我很好，謝謝！"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(name="Bob", user_input="你叫什麼名字？")
print(chat_llm.invoke(messages).content)

<s>你是一個擁有強大能力的 AI 機器人。你的名字是Bob。   [INST] 你好，你好嗎？ [/INST] 我很好，謝謝！ [INST] 你叫什麼名字？ [/INST] 我叫 Bob。


### 2.3 混合使用

In [ ]:
system_prompt = "你是一個擁有強大能力的 AI 機器人。"
question_prompt = "告訴我關於一個{content}的{adjective}笑話。"

full_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_prompt),
    HumanMessage("你好，你好嗎？"),
    AIMessage("我很好，謝謝！"),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=question_prompt,
            input_variables=["content", "adjective"])
    )
])
full_prompt.pretty_print()

================================ System Message ================================

你是一個擁有強大能力的 AI 機器人。

================================ Human Message =================================

你好，你好嗎？

================================== Ai Message ==================================

我很好，謝謝！

================================ Human Message =================================

告訴我關於一個{content}的{adjective}笑話。


## 3. LLM Chain
對於簡單的任務，使用單一 LLM（大型語言模型）效果很好。然而，對於更複雜的任務，通常需要鍊式多個步驟和/或模型。

在LangChain中，可以使用傳統的 LLMChain，較新且建議的方法是 LangChain 表達式語言（LCEL）。

### 3.1 LLMChain

In [ ]:
template = """
問題: {question}

Let's think step by step.

答案:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "NBA 2023 年總冠軍球隊是誰?"
print(llm_chain.invoke({"question": question})["text"])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(



問題: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

答案: 
目前尚未決定，需等到 NBA 2023 賽季結束後才會有結果。


### 3.2 LCEL（LangChain Expression Language）

In [ ]:
# 需要加入 Instruction 參數，才能告訴模型你的對話結束了，因目前使用的模型上不支援 chat 模式
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。",),
        ("human", "{question}"),
    ]
)

# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | StrOutputParser()

# 將問題透過參數化的方式帶入
print(llm_chain.invoke({"question": "機器學習和深度學習有什麼不同？"}))

<s>你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。   [INST] 機器學習和深度學習有什麼不同？ [/INST] 機器學習 (Machine Learning) 和深度學習 (Deep Learning) 是人工智慧領域中的兩個重要分支。它們之間存在一些差異，但同時也具有高度相關性。

1. 概念：機器學習是一種計算機科學方法，旨在使電腦系統能自動從數據中獲得經驗，從而改善其表現。它涉及到如何讓電腦在沒有明確指示的情況下進行操作。深度學習則是機器學習的一個子集，主要利用多層次神經網絡模型來解決複雜的結構化或非結構化的數據分析任務。

2. 模型：機器學習涵蓋了多種算法，如支持向量機、決策樹等，而深度學習則主要指的是以卷積神經網路（CNN）、反復神经网络（RNN）等為代表的深層层人工智能模型。

3. 應用：機器學習可以處理各種類型的數據，包括結構化數據、非結構化數據以及混合型數據。深度學習通常適用於圖像識別、自然語言理解、語音辨識等需要處理大量數據且具有一定規則性的任務。

4. 難度：深度學習需要更多的數據和運算資源，尤其是在訓練模型時。然而，隨著GPU等硬件技術的進步，深度學習已越來越容易被廣泛使用。

5. 未來：深度學習目前正在快速發展，在很多領域都取得了顯著成果。但是，機器學習仍然是人工智能研究的核心部分，深度學習只是其中的一部分。因此，機器學習將繼續保持其重要地位，而深度學習則會進一步拓展至其他領域。
